In [ ]:
git clone https://github.com/OpenRLHF/OpenRLHF.git
cd OpenRLHF

bash examples/scripts/nvidia_docker_install.sh

In [ ]:
bash examples/scripts/docker_run.sh

# Sau khi image được pull về hoàn tất và container được khởi chạy.
# Hãy chạy các lệnh này bên trong container.

!pip uninstall xgboost transformer_engine flash_attn pynvml -y
!pip install openrlhf[vllm]

In [ ]:
set -x

ray job submit --address="http://127.0.0.1:8265" \
   --runtime-env-json='{"setup_commands": ["pip install openrlhf[vllm]"]}' \
   -- python3 -m openrlhf.cli.train_ppo_ray \
   --ref_num_nodes 1 \
   --ref_num_gpus_per_node 1 \
   --reward_num_nodes 1 \
   --reward_num_gpus_per_node 1 \
   --critic_num_nodes 1 \
   --critic_num_gpus_per_node 1 \
   --actor_num_nodes 1 \
   --actor_num_gpus_per_node 1 \
   --vllm_num_engines 1 \
   --vllm_tensor_parallel_size 1 \
   --colocate_critic_reward \
   --colocate_actor_ref \
   --pretrain DATVO110/Llama-3.2-1B-Instruct-Chat-sft \
   --reward_pretrain DATVO110/Llama-3.2-1B-RM-DPO \
   --save_path /openrlhf/examples/checkpoint/Llama-3.2-1B-RLHF-2k \
   --micro_train_batch_size 4 \
   --train_batch_size 64 \
   --micro_rollout_batch_size 8 \
   --rollout_batch_size 512 \
   --max_epochs 2 \
   --prompt_max_len 1024 \
   --generate_max_len 1024 \
   --zero_stage 3 \
   --bf16 \
   --actor_learning_rate 5e-7 \
   --critic_learning_rate 9e-6 \
   --init_kl_coef 0.01 \
   --prompt_data thuanan/Prompt-Vi-Alpaca-Preference-2k \
   --input_key context_messages \
   --apply_chat_template \
   --normalize_reward \
   --packing_samples \
   --adam_offload \
   --flash_attn \
   --gradient_checkpointing \
   --load_checkpoint \
   --use_wandb f3cd4b8a4236b7892cf09e7c6bd23a7bd8b86dbf

In [ ]:
!pip install huggingface_hub

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

ckpt_path = "OpenRLHF/examples/checkpoint/Llama-3.2-1B-RLHF-2k"
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)
model = AutoModelForCausalLM.from_pretrained(ckpt_path)

In [ ]:
type(model)

In [ ]:
type(tokenizer)

In [ ]:
model.push_to_hub(
    "DATVO110/Llama-3.2-1B-RLHF-2k-vi-alpaca",
    commit_message="Add model ckpt",
)

In [ ]:
tokenizer.push_to_hub(
    "DATVO110/Llama-3.2-1B-RLHF-2k-vi-alpaca",
    commit_message="Add tokenizer ckpt",
)